In [1]:
import os, sys
import numpy as np
from itertools import cycle
import data_loader
import tensorflow as tf
from tensorflow.contrib import rnn
import word2glove as w2g

In [2]:
qa_data = data_loader.load_questions_answers('data')

In [3]:
qa_data.keys()

['training',
 'validation',
 'answer_vocab',
 'question_vocab',
 'max_question_length']

In [4]:
sub_qa_data = {}
sub_qa_data['training'] = qa_data['training'][:10]
sub_qa_data['validation'] = qa_data['validation'][:10]
s

In [5]:
len(sub_qa_data['training'])

10

In [12]:
len(w2g.encode_onehot(0., qa_data['question_vocab'],0))

15182

In [3]:
#Get questions in batches
def one_hot_qa(ques, ans):
    ques_holder = np.eye(len(qa_data['question_vocab'].keys()))
    ans_holder = np.eye(len(qa_data['answer_vocab'].keys()))
    enc_ques = np.array([])
    enc_ans = np.array([])
    
    for word in ques:
        if enc_ques.size == 0:
            enc_ques = ques_holder[int(word)]
        else:
            enc_ques = np.vstack((enc_ques, ques_holder[int(word)]))

    enc_ans = ans_holder[ans]            
    return enc_ques, enc_ans

def get_qa_batches(batch_size):
    ques_arr = np.array([])
    ans_arr = np.array([])
    counter = 0
    
    for entry in qa_data['training']:
        if ques_arr.size == 0:
            ques_arr, ans_arr = one_hot_qa(entry['question'], entry['answer'])
        else:
            next_ques, next_ans = one_hot_qa(entry['question'], entry['answer'])
            ques_arr = np.dstack((ques_arr, next_ques))
            ans_arr = np.vstack((ans_arr, next_ans))
            
        counter += 1
        if counter == batch_size:
            yield np.transpose(ques_arr, (2, 0, 1)), np.transpose(ans_arr, (0, 1))
            counter = 0
            ques_arr = np.array([])
            ans_arr = np.array([])

In [4]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 15182 # MNIST data input (img shape: 28*28)
n_steps = 22 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 1000 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [5]:
def RNN(x, weights, biases):
    print x.get_shape()
    x = tf.unstack(x, n_steps, 1)
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# # Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# # Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    train_data_generator = get_qa_batches(128)
    itr = 1
    while itr < 10000:
        itr += 1
        batch_question,batch_answer = train_data_generator.next()
        sess.run(optimizer, feed_dict={x: batch_question, y: batch_answer})
        acc = sess.run(accuracy, feed_dict={x: batch_question, y: batch_answer})
        loss = sess.run(cost, feed_dict={x: batch_question, y: batch_answer})
        print "Iter " + str(itr) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)


(?, 22, 15182)
Iter 2, Minibatch Loss= 6.661595, Training Accuracy= 0.00000
Iter 3, Minibatch Loss= 9.913521, Training Accuracy= 0.34375
Iter 4, Minibatch Loss= 6.378217, Training Accuracy= 0.00000
Iter 5, Minibatch Loss= 6.724869, Training Accuracy= 0.00000
Iter 6, Minibatch Loss= 6.659103, Training Accuracy= 0.00000
Iter 7, Minibatch Loss= 6.684987, Training Accuracy= 0.03906
Iter 8, Minibatch Loss= 6.183796, Training Accuracy= 0.15625
Iter 9, Minibatch Loss= 7.943097, Training Accuracy= 0.22656
Iter 10, Minibatch Loss= 5.878778, Training Accuracy= 0.21094
Iter 11, Minibatch Loss= 6.150809, Training Accuracy= 0.01562
Iter 12, Minibatch Loss= 6.257290, Training Accuracy= 0.00781
Iter 13, Minibatch Loss= 6.238634, Training Accuracy= 0.00000
Iter 14, Minibatch Loss= 6.305418, Training Accuracy= 0.00000
Iter 15, Minibatch Loss= 6.109542, Training Accuracy= 0.02344
Iter 16, Minibatch Loss= 5.931391, Training Accuracy= 0.04688
Iter 17, Minibatch Loss= 5.627723, Training Accuracy= 0.05469
I

KeyboardInterrupt: 